# Performance Manual openai

In [ ]:
import json
import pandas as pd
 
# Opening JSON file
f = open('../openai.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
import os
import openai
openai.api_key =data['key']

def get_topic_openai(review):
    prompt = '''

    Please identify which topics the client is touching in the review

    Possible Topics
    
    1. Longevity
    2. Material and Quality
    3. Fit and Comfort
    4. Versatility
    5. User Experience
    6. Price and Value
    7. Brand Reputation
    8. Packaging and Presentation
    9. Customer Service
    10. Design 
    
    Review:
    
    {}
    
    Topics:
     
    '''.format(review)

    aux = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt.format(row['Review']),
      max_tokens=200,
      temperature=1
    )


    if 'None' not in aux.to_dict_recursive()['choices'][0]['text']:

        topics = [s.split('.')[1].strip()  for s in aux.to_dict_recursive()['choices'][0]['text'].split('\n') if s!= '']
    else:
        topics = []
        
    return topics



review = 'Thanks!'

get_topic_openai(review)

    
    

In [ ]:
guess = [] 
recalls = []
precisions = []
for index, row in manual.iterrows():
    print(index)
    my_guess = get_topic_openai(row['Review'])
    real = list(row.dropna().index)
    if len(real)>1:
        real = real[1:]
        real = list(row.dropna().index)[1:] 
        recall = pd.DataFrame(real)
        recall.columns = ['topic']
        recall['value'] = [ t in my_guess for t in real]
        recalls.append(recall)
    
    if len(my_guess)>0:
        precision = pd.DataFrame(my_guess)
        precision.columns = ['topic']
        precision['value'] = [ t in real for t in my_guess]
        precisions.append(precision)

recall = pd.concat(recalls).groupby(['topic']).mean().reset_index()
precision = pd.concat(precisions).groupby(['topic']).mean().reset_index()

metrics = recall.merge(precision, on = ['topic'])

metrics.columns = ['topic','recall','precision']

metrics